## Create feast registry

In [1]:
!../../feast_env/bin/feast apply

/home/hoang/Documents/mlops-labs/feature-store/feast_env/lib/python3.8/site-packages/pkg_resources/_vendor/pyparsing.py:943: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  collections.MutableMapping.register(ParseResults)
Created entity event_id
Created feature view mnist_fresh_feature_view
Created feature view mnist_feature_view
Created feature service mnist_feature_v1
Created feature service mnist_feature_v2

Deploying infrastructure for mnist_fresh_feature_view
Deploying infrastructure for mnist_feature_view


## Listing entities, featureView, featureService

In [2]:
!../../feast_env/bin/feast entities list

/home/hoang/Documents/mlops-labs/feature-store/feast_env/lib/python3.8/site-packages/pkg_resources/_vendor/pyparsing.py:943: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  collections.MutableMapping.register(ParseResults)
NAME      DESCRIPTION    TYPE
event_id  event id       ValueType.UNKNOWN


In [3]:
!../../feast_env/bin/feast feature-views list

/home/hoang/Documents/mlops-labs/feature-store/feast_env/lib/python3.8/site-packages/pkg_resources/_vendor/pyparsing.py:943: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  collections.MutableMapping.register(ParseResults)
NAME                      ENTITIES      TYPE
mnist_fresh_feature_view  {'event_id'}  FeatureView
mnist_feature_view        {'event_id'}  FeatureView


In [4]:
!../../feast_env/bin/feast feature-services list

/home/hoang/Documents/mlops-labs/feature-store/feast_env/lib/python3.8/site-packages/pkg_resources/_vendor/pyparsing.py:943: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  collections.MutableMapping.register(ParseResults)
NAME              FEATURES
mnist_feature_v1  mnist_feature_view:feature
mnist_feature_v2  mnist_fresh_feature_view:feature


In [5]:
!../../feast_env/bin/feast data-sources list

/home/hoang/Documents/mlops-labs/feature-store/feast_env/lib/python3.8/site-packages/pkg_resources/_vendor/pyparsing.py:943: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  collections.MutableMapping.register(ParseResults)
NAME                   CLASS
mnist_push_source      <class 'feast.data_source.PushSource'>
mnist_postgres_source  <class 'feast.infra.offline_stores.contrib.postgres_offline_store.postgres_source.PostgreSQLSource'>


## Retrieving features

In [8]:
import pandas as pd
import json
import pickle
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
from feast import FeatureStore

In [9]:
from feast import RepoConfig, FeatureStore
from feast.repo_config import RegistryConfig
from feast.infra.online_stores.redis import RedisOnlineStoreConfig
from feast.infra.offline_stores.contrib.postgres_offline_store.postgres import PostgreSQLOfflineStoreConfig

PSQL_HOST = "localhost"
PSQL_PORT = 5432
PSQL_USER = "postgres"
PSQL_PASSWORD = "mysecretpassword"
PSQL_DB = "feast"

REDIS_HOST = "localhost"
REDIS_PORT = 6379

repo_config = RepoConfig(
    registry=RegistryConfig(
        registry_type="sql",
        path=f"postgresql://{PSQL_USER}:{PSQL_PASSWORD}@{PSQL_HOST}:{PSQL_PORT}/{PSQL_DB}",
    ),
    project="feast_demo_local",
    provider="local",
    offline_store=PostgreSQLOfflineStoreConfig(host=PSQL_HOST, port=PSQL_PORT, database=PSQL_DB,
                                              user=PSQL_USER, password=PSQL_PASSWORD),
    online_store=RedisOnlineStoreConfig(connection_string=f"{REDIS_HOST}:{REDIS_PORT}"),
    entity_key_serialization_version=2,
)

store = FeatureStore(config=repo_config)

In [14]:
entity_df = pd.DataFrame([{"event_id": i} for i in range(10)])
entity_df["event_timestamp"] = pd.to_datetime('now', utc=True)
entity_df

,event_id,event_timestamp
0,0,2022-12-13 17:12:57.811939+00:00
1,1,2022-12-13 17:12:57.811939+00:00
2,2,2022-12-13 17:12:57.811939+00:00
3,3,2022-12-13 17:12:57.811939+00:00
4,4,2022-12-13 17:12:57.811939+00:00
5,5,2022-12-13 17:12:57.811939+00:00
6,6,2022-12-13 17:12:57.811939+00:00
7,7,2022-12-13 17:12:57.811939+00:00
8,8,2022-12-13 17:12:57.811939+00:00
9,9,2022-12-13 17:12:57.811939+00:00


In [15]:
# Retrieving from the offline store with a feature service v1
feature_v1 = store.get_feature_service("mnist_feature_v1")
training_data = store.get_historical_features(features=feature_v1, entity_df=entity_df)
training_data = training_data.to_df()
training_data

,event_id,event_timestamp,feature
0,0,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,1,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,2,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,3,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,4,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
5,5,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
6,6,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
7,7,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
8,8,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
9,9,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [16]:
feature_v2 = store.get_feature_service("mnist_feature_v2")
training_data = store.get_historical_features(features=feature_v2, entity_df=entity_df)
training_data.to_df()

,event_id,event_timestamp,feature
0,0,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,1,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,2,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,3,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,4,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
5,5,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
6,6,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
7,7,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
8,8,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
9,9,2022-12-13 17:12:57.811939,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


## Push data to feast

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

with np.load("data/mnist.npz") as data:
    train_examples = data['x_train'].astype(np.float64)
    train_labels = data['y_train'].astype(np.int64)

print(train_examples.shape, train_labels.shape)

(60000, 28, 28) (60000,)


In [22]:
data = []
for i in range(len(train_examples)):
    if i == 100:
        break
    example = json.dumps(train_examples[i].tolist())
    d = {}
    d['feature'] = example
#     for j in range(len(example)):
#         d[f'feature_{j}'] = example[j]
    d['label'] = train_labels[i]
    d['event_id'] = i + 1000
    data.append(d)

In [23]:
data_df = pd.DataFrame(data)

In [24]:
# Creating timestamps for the data
data_df["event_timestamp"] = pd.Timestamp.now()
data_df

,feature,label,event_id,event_timestamp
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",5,1000,2022-12-14 00:13:54.295681
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0,1001,2022-12-14 00:13:54.295681
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",4,1002,2022-12-14 00:13:54.295681
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1,1003,2022-12-14 00:13:54.295681
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",9,1004,2022-12-14 00:13:54.295681
...,...,...,...,...
95,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0,1095,2022-12-14 00:13:54.295681
96,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",7,1096,2022-12-14 00:13:54.295681
97,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",8,1097,2022-12-14 00:13:54.295681
98,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",3,1098,2022-12-14 00:13:54.295681


In [25]:
from feast.data_source import PushMode
store.push("mnist_push_source", data_df, to=PushMode.ONLINE)

## Make online features

In [26]:
store.materialize_incremental(end_date=datetime.now())

Materializing 2 feature views to 2022-12-14 00:14:02+07:00 into the redis online store.

mnist_fresh_feature_view from 1995-07-28 17:14:02+07:00 to 2022-12-14 00:14:02+07:00:


100%|███████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 5987.67it/s]


mnist_feature_view from 1995-07-28 17:14:02+07:00 to 2022-12-14 07:14:02+07:00:


100%|███████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 5358.97it/s]


## Fetch online features

In [27]:
features = store.get_online_features(features=feature_v1,
                                     entity_rows=[{"event_id": i} for i in range(1000)]).to_dict()

features_df = pd.DataFrame.from_dict(data=features)
features_df.dropna(inplace=True)
features_df

,event_id,feature
0,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...
95,95,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
96,96,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
97,97,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
98,98,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [28]:
!docker exec -it feast_redis redis-cli keys '*'

  1) "\x02\x00\x00\x00event_id\x04\x00\x00\x00\b\x00\x00\x00@\x04\x00\x00\x00\x00\x00\x00feast_demo_local"
  2) "\x02\x00\x00\x00event_id\x04\x00\x00\x00\b\x00\x00\x00\a\x00\x00\x00\x00\x00\x00\x00feast_demo_local"
  3) "\x02\x00\x00\x00event_id\x04\x00\x00\x00\b\x00\x00\x00Z\x00\x00\x00\x00\x00\x00\x00feast_demo_local"
  4) "\x02\x00\x00\x00event_id\x04\x00\x00\x00\b\x00\x00\x00\x18\x04\x00\x00\x00\x00\x00\x00feast_demo_local"
  5) "\x02\x00\x00\x00event_id\x04\x00\x00\x00\b\x00\x00\x002\x00\x00\x00\x00\x00\x00\x00feast_demo_local"
  6) "\x02\x00\x00\x00event_id\x04\x00\x00\x00\b\x00\x00\x00\xed\x03\x00\x00\x00\x00\x00\x00feast_demo_local"
  7) "\x02\x00\x00\x00event_id\x04\x00\x00\x00\b\x00\x00\x00I\x04\x00\x00\x00\x00\x00\x00feast_demo_local"
  8) "\x02\x00\x00\x00event_id\x04\x00\x00\x00\b\x00\x00\x000\x00\x00\x00\x00\x00\x00\x00feast_demo_local"
  9) "\x02\x00\x00\x00event_id\x04\x00\x00\x00\b\x00\x00\x00=\x00\x00\x00\x00\x00\x00\x00feast_demo_local"
 10) "\x02\x00\x00\x00event_id

**Note:** PushMode (100 events) + materialize (100 events) = 200 events